In [1]:
import torch
import transformers
import pandas as pd
import fire


/opt/homebrew/Caskroom/miniconda/base/envs/tlab_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(fire.__version__)

0.5.0


# 0. Подготовим список тайтлов фильмов

https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

Возьмем список из 1000 фильмов отсюда, чтобы не выписывать их самостоятельно.

In [10]:
df = pd.read_csv("../data/imdb_top_1000.csv")

In [11]:
df.head(5)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [16]:
names = list(df.Series_Title.unique())

In [18]:
with open('../data/titles.txt', 'w') as f:
        f.write('\n'.join(names))

# 1. Подготовим промпты

Можем использовать достаточно большой список шаблонов для промпта, в том числе в явном виде просить её сгенерировать положительный ответ, отрицательный. Однако, мы не будем пытаться использовать длинные шаблоны промптов, потому что gpt-2 достаточно маленькая, длинные промпты будут плохо действовать на маленьких моделях, и это будет игра с нулевой суммой.

Возможно, я неправа в том, что я нарочно прошу генерировать позитивные отзывы напрямую в промпте.

In [22]:
import json
import random
import numpy as np

In [58]:
TEMPLATE_SRC = "../data/templates.json"
TITLES_SRC = "../data/titles.txt"
PROMPTS_SRC = "../data/prompts.txt"

In [54]:
import sys
sys.path.append('../')
from src.make_dataset import prepate_prompts, prepare_dataset

In [61]:
train_prompts = prepate_prompts(300, TEMPLATE_SRC, TITLES_SRC)
test_prompts = prepate_prompts(100, TEMPLATE_SRC, TITLES_SRC)

In [62]:
with open('../data/prompts.txt', 'w') as f:
    f.write('\n'.join(train_prompts))

In [63]:
with open('../data/prompts_test.txt', 'w') as f:
    f.write('\n'.join(test_prompts))

# 2. Сгенерируем данные при помощи gpt-2

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

gen_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb", device_map=device)
gen_model = transformers.AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb", device_map=device)
if gen_tokenizer.pad_token is None:
    gen_tokenizer.pad_token = gen_tokenizer.eos_token

reward_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb", device_map=device)

pytorch_model.bin: 100%|██████████| 548M/548M [00:22<00:00, 24.1MB/s] 
tokenizer_config.json: 100%|██████████| 333/333 [00:00<00:00, 1.01MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 3.52MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 10.9MB/s]
config.json: 100%|██████████| 735/735 [00:00<00:00, 965kB/s]
pytorch_model.bin: 100%|██████████| 268M/268M [00:09<00:00, 27.0MB/s] 


In [3]:
with open("../data/prompts.txt", "r") as f:
    prompts = f.read().split("\n")

In [11]:
torch.mps.empty_cache()

In [ ]:
input = gen_tokenizer(prompts,
                      return_tensors="pt",
                      padding=True).to(device)

output = gen_model.generate(**input,
                            max_new_tokens=60,
                            no_repeat_ngram_size=2,
                            do_sample=True,
                            temperature=0.25,
                            num_return_sequences=2)

In [14]:
output.shape

torch.Size([600, 50])

In [17]:
!pip install scipy

import scipy

print(scipy.__version__)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 46.5 MB/s eta 0:00:0000:0100:01
1.11.4
